# Proposed changes

* Drop (empty) multi-backend promise
* Make all Models and Modules Pytorch-compatible out-of-the box
* Provide training/fine-tuning examples using Pytorch Lightning
* Neural Types are now optional and can do more when used

## 1. NeMo/Jarvis fine-tuning example

Every Jarvis service will have a corresponding script (or more if several models are used) to fine-tune its models on the user data

In [8]:
import pytorch_lightning as pl
import nemo.collections.asr as nemo_asr

#### Method 1 - get model and pretrained weights from NVIDIA NGC cloud

In [9]:
# asr_model = nemo_asr.models.ASRConvCTCModel2.from_cloud(name="QuartzNet15x5-En")

#### Method 2 - get model and pretrained weights from local file

In [10]:
# asr_model = nemo_asr.models.ASRConvCTCModel2.restore_from("my_asr.nemo")

#### Method 3 - instantiate model from config (no pre-trained weights)

In [11]:
# asr_model = nemo_asr.models.ASRConvCTCModel2.from_config("quartznet.yaml")

#### Method 4 - instantiate model using constructor

In [12]:
from ruamel.yaml import YAML
yaml = YAML(typ="safe")
with open('/Users/okuchaiev/repos/NeMo/examples/asr/configs/jasper_an4-2.yaml') as f:
    model_config = yaml.load(f)
asr_model = nemo_asr.models.ASRConvCTCModel2(
    preprocessor_params=model_config['AudioToMelSpectrogramPreprocessor'],
    encoder_params=model_config['JasperEncoder'],
    decoder_params=model_config['JasperDecoder'],
)

[NeMo I 2020-06-15 22:54:19 features:154] PADDING: 16
[NeMo I 2020-06-15 22:54:19 features:162] STFT using conv
[NeMo I 2020-06-15 22:54:19 helpers:66] Instantiated a new Neural Module of type AudioToMelSpectrogramPreprocessor2
[NeMo I 2020-06-15 22:54:19 helpers:66] Instantiated a new Neural Module of type ConvASREncoder
[NeMo I 2020-06-15 22:54:19 helpers:66] Instantiated a new Neural Module of type ConvASRDecoder


In [13]:
# Setup where your training data is
asr_model.setup_training_data(model_config['AudioToTextDataLayer'])
asr_model.setup_validation_data(model_config['AudioToTextDataLayer_eval'])

[NeMo I 2020-06-15 22:54:19 collections:158] Dataset loaded with 897 files totalling 1.39 hours
[NeMo I 2020-06-15 22:54:19 collections:159] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-06-15 22:54:19 helpers:66] Instantiated a new Neural Module of type AudioToTextDataLayer2
[NeMo I 2020-06-15 22:54:19 collections:158] Dataset loaded with 130 files totalling 0.20 hours
[NeMo I 2020-06-15 22:54:19 collections:159] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-06-15 22:54:19 helpers:66] Instantiated a new Neural Module of type AudioToTextDataLayer2


In [ ]:
# User Pytorch Lightning Trainer object instead of NeuralModuleFactory
# trainer = pl.Trainer(val_check_interval=5, amp_level='O1', precision=16, gpus=2, max_epochs=50, distributed_backend='ddp')
trainer = pl.Trainer(val_check_interval=5, max_epochs=1)
trainer.fit(asr_model)

### Export for deployment with Jarvis

In [15]:
asr_model.save_to('qn.nemo', optimize_for_deployment=True)

TODO: Implement Me
